# Representations of Finite Groups — Notes Companion

**6.7970/8.750 Symmetry and its Application to Machine Learning**

This notebook accompanies the [Representations notes](https://symm4ml.mit.edu/notes/representations). Run these examples to explore irreps, Schur's Lemma, and the regular representation using the `symm4ml` library.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/atomicarchitects/symm4ml-colabs/blob/main/representations_notes_companion.ipynb)

## Setup

In [ ]:
%%capture
!pip install https://symm4ml.mit.edu/_static/symm4ml_s26/symm4ml/symm4ml_latest.zip

In [ ]:
import numpy as np
from symm4ml import groups, linalg, rep

## Reference: $P(3)$ multiplication table

In [ ]:
# Build the P(3) multiplication table from 3D permutation matrices
p3_matrices = groups.permutation_matrices(3)
table = groups.make_multiplication_table(p3_matrices)
print(f"P(3) has {len(table)} elements")
print(f"Multiplication table:\n{table}")

---
## Example: Finding and Verifying Irreps of $P(3)$

The function `rep.infer_irreps(table)` finds all irreducible representations of a group from its multiplication table.
This uses the decomposition algorithm from the [decomposition notes](https://symm4ml.mit.edu/notes/decomposition-and-products) applied to the regular representation.

In [ ]:
# Find all irreps of P(3)
irreps = rep.infer_irreps(table)

print(f"Found {len(irreps)} irreps:")
for i, ir in enumerate(irreps):
    print(f"  \u0393_{i+1}: dimension {ir.shape[1]}")

### Verifying with `is_a_representation` and `is_an_irrep`

We can check that each set of matrices forms a valid representation,
and that each one is irreducible.

In [ ]:
for i, ir in enumerate(irreps):
    is_rep = rep.is_a_representation(table, ir)
    is_irr = rep.is_an_irrep(table, ir)
    print(f"\u0393_{i+1} (dim {ir.shape[1]}): is_a_representation={is_rep}, is_an_irrep={is_irr}")

### Irreps are unique up to similarity transform

As the notes explain, if $D(g)$ is a representation and $U$ is invertible, then $U D(g) U^{-1}$ is also a valid representation.
The function `rep.are_isomorphic` checks if two representations are related by such a change of basis.

In [ ]:
# Take the 2D irrep and apply a random similarity transform
gamma_2d = [ir for ir in irreps if ir.shape[1] == 2][0]

np.random.seed(7)
U, _ = np.linalg.qr(np.random.randn(2, 2))  # random orthogonal matrix
gamma_2d_transformed = rep.similarity_transform(gamma_2d, U)

print(f"Original \u0393_2[1]:\n{gamma_2d[1]}\n")
print(f"Transformed \u0393_2[1]:\n{gamma_2d_transformed[1]}\n")
print(f"Are they isomorphic? {rep.are_isomorphic(gamma_2d, gamma_2d_transformed)}")

---
## Example: The Regular Representation Contains All Irreps

The **regular representation** is built directly from the multiplication table: $D^{\text{reg}}(g)|h\rangle = |gh\rangle$.
It has dimension $h$ (the order of the group) and contains **every** irrep.

In [ ]:
# Build the regular representation
reg = rep.regular_representation(table)
print(f"Regular rep shape: {reg.shape}")
print(f"Dimension = h = {reg.shape[1]}")

In [ ]:
# Decompose the regular representation into irreps
sub_irreps = rep.decompose_rep_into_irreps(reg)

print(f"Regular rep decomposes into {len(sub_irreps)} irreps:")
for i, ir in enumerate(sub_irreps):
    print(f"  Sub-irrep {i+1}: dimension {ir.shape[1]}")

# Identify each sub-irrep
print("\nMatching to known irreps:")
for i, sub in enumerate(sub_irreps):
    for j, known in enumerate(irreps):
        if rep.are_isomorphic(sub, known):
            print(f"  Sub-irrep {i+1} \u2245 \u0393_{j+1} (dim {known.shape[1]})")
            break

In [ ]:
# Verify: each irrep Γ_j appears ℓ_j times, and sum of ℓ_j^2 = h
dims = [ir.shape[1] for ir in irreps]
print(f"Irrep dimensions: {dims}")
print(f"Sum of \u2113_j^2 = {sum(d**2 for d in dims)} = h = {len(table)}")